
# Simple DQN

In [7]:
!uv venv
!uv pip install -r ClusterEnv/pyproject.toml

Using CPython 3.10.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
  × No solution found when resolving dependencies:
  ╰─▶ Because the current Python version (3.10.12) does not satisfy
      Python>=3.11,<4.0 and tianshou>=1.0.0 depends on Python>=3.11,<4.0, we
      can conclude that tianshou>=1.0.0 cannot be used.
      And because only the following versions of tianshou are available:
          tianshou<=1.0.0
          tianshou==1.1.0
      and clusterenv depends on tianshou>=1.0.0, we can conclude that your
      requirements are unsatisfiable.


### Imports

In [1]:
from tianshou.utils.logger.tensorboard import SummaryWriter
import gymnasium as gym
import tianshou as ts
import numpy as np
import torch as th
import typing

ModuleNotFoundError: No module named 'tianshou'

In [ ]:
# TODO:
#  Run For The Following sizes:
#  Machines: 2 Jobs: 2 Resource: 1 time: 1
#  Machines: 2 Jobs: 20 Resource: 1 time: 1
#  Machines: 2 Jobs: 20 Resource: 3 time: 5
#  Machines: 2 Jobs: 20 Resource: 3 time: 10

In [ ]:
from envs.cluster import ClusterWrapper

def scenario_0(q_size: int = 2):
    return ClusterWrapper(
        gym.make(
            "Cluster-v0",
            n_machines=2,
            n_jobs=2,
            n_resource=1,
            max_ticks=1,
            max_episode_steps=10
        ),
        queue_size=q_size,
    )

def scenario_1(q_size: int = 2):
    return ClusterWrapper(
        gym.make(
            "Cluster-v0",
            n_machines=2,
            n_jobs=20,
            n_resource=1,
            max_ticks=1,
            max_episode_steps=100
        ),

        queue_size=q_size,
    )

def scenario_2(q_size: int = 2):
    return ClusterWrapper(
        gym.make(
            "Cluster-v0",
            n_machines=2,
            n_jobs=20,
            n_resource=3,
            max_ticks=5,
            max_episode_steps=1_000
        ),
        queue_size=q_size,
    )

def scenario_3(q_size: int = 2):
    return ClusterWrapper(
        gym.make(
            "Cluster-v0",
            n_machines=2,
            n_jobs=20,
            n_resource=3,
            max_ticks=10,
            max_episode_steps=3_000
        ),
        queue_size=q_size,
    )

In [ ]:
def run(
    model_cls,
    gen_env: typing.Callable[[],gym.Env],
    *,
    lr: float = 1e-3,
    max_epoch: int = 100,
    episode_per_collect: int = 10,
    step_per_epoch: int = 200,
    batch_size: int = 32,
    episode_per_test=1,
    repeat_per_collect=1
):
    env = gen_env()
    model = model_cls(env.observation_space, env.action_space)
    train_envs = ts.env.DummyVectorEnv([gen_env for _ in range(1)])
    test_envs = ts.env.DummyVectorEnv([gen_env for _ in range(1)])
    optimizer = th.optim.Adam(model.parameters(), lr=lr)
    logger = ts.utils.TensorboardLogger(writer=SummaryWriter(f"assets/log/dqn/{type(model).__name__}_{gen_env.__name__}"))
    policy = ts.policy.DQNPolicy(
        model=model,
        optim=optimizer,
        action_space=env.action_space,
    )
    train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(10_000, 10), exploration_noise=True)
    test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)
    val =  ts.trainer.OffpolicyTrainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,
        max_epoch=max_epoch,
        episode_per_collect=episode_per_collect,
        step_per_epoch=step_per_epoch,
        batch_size=batch_size,
        episode_per_test=episode_per_test,
        logger=logger,
        repeat_per_collect=repeat_per_collect,
    ).run()
    th.save(policy.state_dict(), f"assets/policy/{type(model).__name__}_{gen_env.__name__}.pth")
    return val

## Models

In [ ]:
class SimpleNN(th.nn.Module):
    def __init__(self, state: dict, action: dict):
        super().__init__()
        machines = state["machinesAvailability"]
        jobs = state["jobsUsage"]
        combined: int = np.prod(machines.shape) + np.prod(jobs.shape)
        self.model = th.nn.Sequential(
            th.nn.Linear(combined, 64),
            th.nn.ReLU(),
            th.nn.Linear(64, 64),
            th.nn.ReLU(),
            th.nn.Linear(64, action.n),
            th.nn.Softmax(dim=-1),
        )

    def forward(self, obs, state=None, **kwargs):
        if not isinstance(obs, dict):
            obs = {k: th.tensor(v, dtype=th.float) for k, v in obs.items()}
        batch_size = obs["machinesAvailability"].shape[0]
        machines = obs["machinesAvailability"].view(batch_size, -1)
        jobs = obs["jobsUsage"].view(batch_size, -1)
        x = th.cat([machines, jobs], dim=1)
        logits = self.model(x)
        return logits, state

In [ ]:
class SimpleConv(th.nn.Module):
    def __init__(self, state: dict, action: gym.Space, *args, **kwargs):
        super().__init__()
        machines = state["machinesAvailability"]
        jobs = state["jobsUsage"]
        self.in_channels: int = max(jobs.shape[0], machines.shape[0])
        self.embeder = th.nn.Sequential(
            th.nn.Conv2d(in_channels=self.in_channels, out_channels=64, kernel_size=(1, machines.shape[2])),
            th.nn.ReLU(),
            th.nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(jobs.shape[1], 1)),
            th.nn.ReLU(),
            th.nn.Conv2d(in_channels=32, out_channels=16, kernel_size=(1, 1)),
            th.nn.Flatten(),
            th.nn.ReLU(),
            th.nn.Linear(in_features=16, out_features=16),
            th.nn.ReLU(),
        )
        self.clf = th.nn.Sequential(
            th.nn.Linear(in_features=32, out_features=64),
            th.nn.ReLU(),
            th.nn.AvgPool1d(2),
            th.nn.Linear(in_features=32, out_features=action.n),
            th.nn.Softmax()
        )


    def forward(self, obs, state=None, **kwargs):
        if not isinstance(obs, dict):
            obs = {k: th.tensor(v, dtype=th.float) for k, v in obs.items()}
        machines = obs["machinesAvailability"]
        jobs = obs["jobsUsage"]
        machines_emb = self.embeder(machines)
        jobs_emb = self.embeder(jobs)
        x = th.cat([machines_emb, jobs_emb], dim=1)
        logits = self.clf(x)
        return logits, state

## Runs

### Simple NN

In [ ]:
run(SimpleNN, scenario_0)

Epoch #1: 260it [00:00, 585.35it/s, env_step=260, gradient_step=260, len=10, n/ep=10, n/st=100, rew=-2.83]                         

Epoch #1: test_reward: -2.828968 ± 0.000000, best_reward: -2.000000 ± 0.000000 in #0



Epoch #2: 201it [00:00, 623.46it/s, env_step=460, gradient_step=460, len=10, n/ep=10, n/st=100, rew=-2.83]                         

Epoch #2: test_reward: -2.000000 ± 0.000000, best_reward: -2.000000 ± 0.000000 in #0



Epoch #3: 210it [00:00, 539.57it/s, env_step=670, gradient_step=670, len=2, n/ep=10, n/st=20, rew=0.00]                         

...


Epoch #99: 201it [00:00, 474.38it/s, env_step=19870, gradient_step=19870, len=2, n/ep=10, n/st=20, rew=0.00]                         

Epoch #99: test_reward: 0.000000 ± 0.000000, best_reward: 0.000000 ± 0.000000 in #3



Epoch #100: 201it [00:00, 337.14it/s, env_step=20070, gradient_step=20070, len=2, n/ep=10, n/st=20, rew=0.00]                         

Epoch #100: test_reward: 0.000000 ± 0.000000, best_reward: 0.000000 ± 0.000000 in #3


InfoStats(gradient_step=20070, best_reward=0.0, best_reward_std=0.0, train_step=20070, train_episode=9860, test_step=212, test_episode=101, timing=TimingStats(total_time=41.64589500427246, train_time=41.3753387928009, train_time_collect=16.080097675323486, train_time_update=22.51021146774292, test_time=0.2705562114715576, update_speed=485.07155676733885))

In [ ]:
run(SimpleNN, scenario_1)

Epoch #1: 1000it [00:02, 380.99it/s, env_step=1000, gradient_step=1000, len=100, n/ep=10, n/st=1000, rew=-415.30]

Epoch #1: test_reward: -200.723693 ± 0.000000, best_reward: -200.723693 ± 0.000000 in #1



Epoch #2: 390it [00:00, 561.47it/s, env_step=1390, gradient_step=1390, len=39, n/ep=10, n/st=390, rew=-200.72] 

Epoch #2: test_reward: -415.302740 ± 0.000000, best_reward: -200.723693 ± 0.000000 in #1



Epoch #3: 1000it [00:01, 583.32it/s, env_step=2390, gradient_step=2390, len=100, n/ep=10, n/st=1000, rew=-415.30] 

...


Epoch #99: 290it [00:00, 639.48it/s, env_step=68640, gradient_step=68640, len=29, n/ep=10, n/st=290, rew=-214.50] 

Epoch #99: test_reward: -415.302740 ± 0.000000, best_reward: -103.747550 ± 0.000000 in #4



Epoch #100: 1000it [00:01, 518.25it/s, env_step=69640, gradient_step=69640, len=100, n/ep=10, n/st=1000, rew=-415.30] 

Epoch #100: test_reward: -220.138095 ± 0.000000, best_reward: -103.747550 ± 0.000000 in #4


InfoStats(gradient_step=69640, best_reward=-103.7475503527924, best_reward_std=0.0, train_step=69640, train_episode=1000, test_step=6994, test_episode=101, timing=TimingStats(total_time=119.08180403709412, train_time=114.66745591163635, train_time_collect=41.5474009513855, train_time_update=69.21594715118408, test_time=4.414348125457764, update_speed=607.3214012323177))

In [ ]:
run(SimpleNN, scenario_2)

Epoch #1: 650it [00:01, 599.16it/s, env_step=650, gradient_step=650, len=65, n/ep=10, n/st=650, rew=-127.04] 

Epoch #1: test_reward: -149.715201 ± 0.000000, best_reward: -127.040722 ± 0.000000 in #0



Epoch #2: 1150it [00:01, 630.56it/s, env_step=1800, gradient_step=1800, len=115, n/ep=10, n/st=1150, rew=-149.72] 


Epoch #2: test_reward: -149.709417 ± 0.000000, best_reward: -127.040722 ± 0.000000 in #0


Epoch #3: 10000it [00:15, 654.65it/s, env_step=11800, gradient_step=11800, len=1000, n/ep=10, n/st=10000, rew=-149.71] 

...


Epoch #99: 1150it [00:01, 663.99it/s, env_step=697000, gradient_step=697000, len=115, n/ep=10, n/st=1150, rew=-149.72] 


Epoch #99: test_reward: -770.668756 ± 0.000000, best_reward: -127.040722 ± 0.000000 in #0


Epoch #100: 10000it [00:14, 681.61it/s, env_step=707000, gradient_step=707000, len=1000, n/ep=10, n/st=10000, rew=-770.67] 


Epoch #100: test_reward: -770.668756 ± 0.000000, best_reward: -127.040722 ± 0.000000 in #0


InfoStats(gradient_step=707000, best_reward=-127.04072227145149, best_reward_std=0.0, train_step=707000, train_episode=1000, test_step=71700, test_episode=101, timing=TimingStats(total_time=1084.2739489078522, train_time=1044.073079109192, train_time_collect=390.3086099624634, train_time_update=624.7311413288116, test_time=40.20086979866028, update_speed=677.1556648153554))

In [ ]:
run(SimpleNN, scenario_3)

Epoch #1: 30000it [00:49, 611.32it/s, env_step=30000, gradient_step=30000, len=3000, n/ep=10, n/st=30000, rew=-171.67] 

Epoch #1: test_reward: -152.675397 ± 0.000000, best_reward: -152.675397 ± 0.000000 in #1



Epoch #2: 2100it [00:03, 684.46it/s, env_step=32100, gradient_step=32100, len=210, n/ep=10, n/st=2100, rew=-152.68] 

Epoch #2: test_reward: -152.675397 ± 0.000000, best_reward: -152.675397 ± 0.000000 in #1



Epoch #3: 2100it [00:03, 670.87it/s, env_step=34200, gradient_step=34200, len=210, n/ep=10, n/st=2100, rew=-152.68] 


...

Epoch #99: 30000it [00:43, 682.75it/s, env_step=2272500, gradient_step=2272500, len=3000, n/ep=10, n/st=30000, rew=-1130.56] 


Epoch #99: test_reward: -171.674998 ± 0.000000, best_reward: -152.675397 ± 0.000000 in #1


Epoch #100: 30000it [00:44, 670.44it/s, env_step=2302500, gradient_step=2302500, len=3000, n/ep=10, n/st=30000, rew=-171.67] 

Epoch #100: test_reward: -152.675397 ± 0.000000, best_reward: -152.675397 ± 0.000000 in #1


InfoStats(gradient_step=2302500, best_reward=-152.6753965638624, best_reward_std=0.0, train_step=2302500, train_episode=1000, test_step=230460, test_episode=101, timing=TimingStats(total_time=3495.8948669433594, train_time=3369.6520574092865, train_time_collect=1254.5685031414032, train_time_update=2024.8962972164154, test_time=126.24280953407288, update_speed=683.304970593981))

### Conv NN

In [ ]:
run(SimpleConv, scenario_0)

Epoch #1: 201it [00:01, 101.52it/s, env_step=200, gradient_step=200, len=10, n/ep=10, n/st=100, rew=-5.86]                         

Epoch #1: test_reward: -2.828968 ± 0.000000, best_reward: -2.828968 ± 0.000000 in #0



Epoch #2: 201it [00:02, 96.27it/s, env_step=400, gradient_step=400, len=2, n/ep=10, n/st=20, rew=0.00]                             

Epoch #2: test_reward: 0.000000 ± 0.000000, best_reward: 0.000000 ± 0.000000 in #2



Epoch #3: 250it [00:02, 85.51it/s, env_step=650, gradient_step=650, len=10, n/ep=10, n/st=100, rew=-2.83]                       

...

Epoch #99: 201it [00:01, 102.92it/s, env_step=20060, gradient_step=20060, len=2, n/ep=10, n/st=20, rew=0.00]                         

Epoch #99: test_reward: 0.000000 ± 0.000000, best_reward: 0.000000 ± 0.000000 in #2



Epoch #100: 201it [00:01, 100.55it/s, env_step=20260, gradient_step=20260, len=2, n/ep=10, n/st=20, rew=0.00]                         


Epoch #100: test_reward: 0.000000 ± 0.000000, best_reward: 0.000000 ± 0.000000 in #2


InfoStats(gradient_step=20260, best_reward=0.0, best_reward_std=0.0, train_step=20260, train_episode=9660, test_step=220, test_episode=101, timing=TimingStats(total_time=211.7680230140686, train_time=211.44733023643494, train_time_collect=20.043417930603027, train_time_update=188.40189242362976, test_time=0.320692777633667, update_speed=95.81582315248811))

In [ ]:
run(SimpleConv, scenario_1)

Epoch #1: 1000it [00:09, 105.83it/s, env_step=1000, gradient_step=1000, len=100, n/ep=10, n/st=1000, rew=-103.75] 

Epoch #1: test_reward: -200.723693 ± 0.000000, best_reward: -103.747550 ± 0.000000 in #0



Epoch #2: 390it [00:03, 100.70it/s, env_step=1390, gradient_step=1390, len=39, n/ep=10, n/st=390, rew=-200.72]

Epoch #2: test_reward: -415.302740 ± 0.000000, best_reward: -103.747550 ± 0.000000 in #0



Epoch #3: 1000it [00:10, 91.56it/s, env_step=2390, gradient_step=2390, len=100, n/ep=10, n/st=1000, rew=-415.30]  

...


Epoch #99: 390it [00:03, 110.31it/s, env_step=67890, gradient_step=67890, len=39, n/ep=10, n/st=390, rew=-200.72] 

Epoch #99: test_reward: -200.723693 ± 0.000000, best_reward: -103.747550 ± 0.000000 in #0



Epoch #100: 390it [00:03, 107.67it/s, env_step=68280, gradient_step=68280, len=39, n/ep=10, n/st=390, rew=-200.72] 

Epoch #100: test_reward: -200.723693 ± 0.000000, best_reward: -103.747550 ± 0.000000 in #0


InfoStats(gradient_step=68280, best_reward=-103.7475503527924, best_reward_std=0.0, train_step=68280, train_episode=1000, test_step=6867, test_episode=101, timing=TimingStats(total_time=703.9502420425415, train_time=698.1019577980042, train_time_collect=51.14694666862488, train_time_update=642.0736031532288, test_time=5.8482842445373535, update_speed=97.80806261505546))

In [ ]:
run(SimpleConv, scenario_2)

Epoch #1: 10000it [01:39, 100.84it/s, env_step=10000, gradient_step=10000, len=1000, n/ep=10, n/st=10000, rew=-770.67] 

Epoch #1: test_reward: -149.715201 ± 0.000000, best_reward: -149.715201 ± 0.000000 in #1



Epoch #2: 1150it [00:12, 92.57it/s, env_step=11150, gradient_step=11150, len=115, n/ep=10, n/st=1150, rew=-149.72]  

Epoch #2: test_reward: -149.715201 ± 0.000000, best_reward: -149.715201 ± 0.000000 in #1



Epoch #3: 1150it [00:10, 106.20it/s, env_step=12300, gradient_step=12300, len=115, n/ep=10, n/st=1150, rew=-149.72] 


...

Epoch #99: 10000it [01:35, 104.47it/s, env_step=892200, gradient_step=892200, len=1000, n/ep=10, n/st=10000, rew=-770.67] 


Epoch #99: test_reward: -770.668756 ± 0.000000, best_reward: -132.689583 ± 0.000000 in #23


Epoch #100: 10000it [01:36, 103.44it/s, env_step=902200, gradient_step=902200, len=1000, n/ep=10, n/st=10000, rew=-770.67] 


Epoch #100: test_reward: -770.668756 ± 0.000000, best_reward: -132.689583 ± 0.000000 in #23


InfoStats(gradient_step=902200, best_reward=-132.68958266883274, best_reward_std=0.0, train_step=902200, train_episode=1000, test_step=91220, test_episode=101, timing=TimingStats(total_time=9085.453593969345, train_time=9009.720768928528, train_time_collect=728.376672744751, train_time_update=8226.125409126282, test_time=75.73282504081726, update_speed=100.13628869735696))

In [ ]:
run(SimpleConv, scenario_3)

Epoch #1: 30000it [05:21, 93.20it/s, env_step=30000, gradient_step=30000, len=3000, n/ep=10, n/st=30000, rew=-1130.56]  


Epoch #1: test_reward: -1130.555816 ± 0.000000, best_reward: -1130.555816 ± 0.000000 in #0


Epoch #2: 30000it [05:45, 86.95it/s, env_step=60000, gradient_step=60000, len=3000, n/ep=10, n/st=30000, rew=-1130.56]  


Epoch #2: test_reward: -171.674998 ± 0.000000, best_reward: -171.674998 ± 0.000000 in #2


Epoch #3: 30000it [06:05, 81.97it/s, env_step=90000, gradient_step=90000, len=3000, n/ep=10, n/st=30000, rew=-171.67]  

...

Epoch #99: 2100it [16:17:10, 27.92s/it, env_step=512900, gradient_step=512900, len=210, n/ep=10, n/st=2100, rew=-152.68] 


Epoch #99: test_reward: -152.675397 ± 0.000000, best_reward: -124.941988 ± 0.000000 in #14


Epoch #100: 2100it [00:29, 70.07it/s, env_step=515000, gradient_step=515000, len=210, n/ep=10, n/st=2100, rew=-152.68] 

Epoch #100: test_reward: -152.675397 ± 0.000000, best_reward: -124.941988 ± 0.000000 in #14


InfoStats(gradient_step=515000, best_reward=-124.9419882449513, best_reward_std=0.0, train_step=515000, train_episode=1000, test_step=51710, test_episode=101, timing=TimingStats(total_time=64592.63916897774, train_time=64550.37857961655, train_time_collect=432.4592430591583, train_time_update=64085.09772801399, test_time=42.260589361190796, update_speed=7.978264594758311))